# 合并数据_汇总：深调套利收入
读取 `data_out` 中的交易量价数据，并按照提供的公式计算 `深调套利收入`。

In [2]:
from pathlib import Path
import pandas as pd

OUTPUT_PATH = Path('../data_output/合并数据_汇总.xlsx')
SUMMARY_KEY = '合并数据_汇总'
SOURCE_KEY = '交易量价数据信息'

if not OUTPUT_PATH.exists():
    raise FileNotFoundError(f'找不到合并后的输出文件: {OUTPUT_PATH}')

data_out = pd.read_excel(OUTPUT_PATH, sheet_name=None)
sheet_order = list(data_out.keys())

if SUMMARY_KEY not in data_out:
    if SOURCE_KEY not in data_out:
        raise KeyError('在工作簿中找不到 `交易量价数据信息` 表，无法创建合并数据。')
    data_out[SUMMARY_KEY] = data_out[SOURCE_KEY].copy()
    sheet_order.append(SUMMARY_KEY)

print(f'已读取 {len(data_out)} 张表，目标: {SUMMARY_KEY}')


已读取 4 张表，目标: 合并数据_汇总


In [10]:
summary_df = data_out[SUMMARY_KEY].copy()
required_columns = ['日前中标出力', '省内中长期上网电量', '日前出清节点价格', '省内中长期均价']
missing_columns = [col for col in required_columns if col not in summary_df.columns]
if missing_columns:
    raise ValueError(f'{SUMMARY_KEY} 缺少以下列: {missing_columns}')

def to_numeric(series):
    return pd.to_numeric(series, errors='coerce')

bid_power = to_numeric(summary_df['日前中标出力'])
contract_power = to_numeric(summary_df['省内中长期上网电量']+summary_df['省间中长期上网电量'])
intra_prov_contact_power = to_numeric(summary_df['省内中长期上网电量'])
inter_prov_contact_power= to_numeric(summary_df['省间中长期上网电量'])
spot_price = to_numeric(summary_df['日前出清节点价格'])
intra_prov_contact_avg_price = to_numeric(summary_df['省内中长期均价'])
inter_prov_contact_avg_price = to_numeric(summary_df['省间中长期均价'])
contract_price= (intra_prov_contact_power * intra_prov_contact_avg_price + inter_prov_contact_power * inter_prov_contact_avg_price) / contract_power

condition = (bid_power < contract_power * 4) & (spot_price < contract_price)
spread = (contract_power * 4 - bid_power) * (contract_price - spot_price) / 4
summary_df['深调套利收入'] = spread.where(condition, 0).fillna(0)

data_out[SUMMARY_KEY] = summary_df
if SOURCE_KEY in data_out and SOURCE_KEY != SUMMARY_KEY:
    data_out[SOURCE_KEY] = summary_df

summary_df[['省内中长期上网电量', '省内中长期均价', '日前中标出力', '日前出清节点价格', '深调套利收入']].head()

,省内中长期上网电量,省内中长期均价,日前中标出力,日前出清节点价格,深调套利收入
0,48.903,312.776220,200.0,280.0,0.0
1,48.900,312.776181,200.0,280.0,0.0
2,48.905,312.776246,200.0,278.0,0.0
3,48.906,312.756003,200.0,278.0,0.0
4,48.512,286.486787,200.0,270.0,0.0


In [12]:
with pd.ExcelWriter(OUTPUT_PATH, engine='openpyxl') as writer:
    for sheet_name in sheet_order:
        frame = data_out[sheet_name]
        frame.to_excel(writer, sheet_name=sheet_name, index=False)

print('已写回 data_output/合并数据_汇总.xlsx，包含新增的深调套利收入列。')

已写回 data_output/合并数据_汇总.xlsx，包含新增的深调套利收入列。
